# Project M2 - Cell Type Classification with Morphology features
### The objective of this project is to classify two cell types (spiny/aspiny) according to their Morphology features using both logistic regression and neural network.
#### The data set is downloaded from the __[Allen Institute data base](http://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html#Computing-Electrophysiology-Features)__ and is already saved in the file "MorphFeatures.csv".
#### Train the classifiers similarly to Project M1 using only morphology featrues. Then try to combine them with electrophysiology features to see how that would change the accuracy.

## Getting started with the Allen's data set
Use python library Pandas to read the csv file. The data set is now stored in Pandas dataframe.

In [121]:
import numpy as np
import pandas as pd

df = pd.read_csv("MorphFeatures.csv",index_col=0)
print(df.shape)
df.head(5) #show some lines on the data from the first record.

(670, 31)


,average_bifurcation_angle_local,average_bifurcation_angle_remote,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,hausdorff_dimension,id,max_branch_order,max_euclidean_distance,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,NaN,0.864267,0.345092,20.723077,0.964510,NaN,491119743,6.0,99.779724,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,NaN,0.903890,0.634047,105.277778,0.862183,NaN,546781359,3.0,432.383110,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,NaN,0.863104,0.417929,73.666667,0.926633,NaN,537042261,6.0,373.630444,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,NaN,0.900537,0.400396,95.979167,0.942049,NaN,689123605,11.0,943.382549,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,NaN,0.873518,0.227626,47.535714,1.000000,NaN,657879305,5.0,186.218009,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


The cell type is determined by the dendrite type in the last column of the data set. Ignore the samples of minority type called "sparsely spiny".

In [122]:
df.dropna(axis=1,inplace=True) # Drop columns with Nan values
df = df.drop_duplicates(subset=['specimen_id']) # drop duplicated of specimen_id

# Get rid of sparsely spiny cells
df = df[df.dendrite_type!='sparsely spiny'] #keep all the data that 'dendrite_type' is not 'sparsely spiny'
print(df.shape)
print(df.columns)
df.head(5)

(619, 29)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'dendrite_type'],
      dtype='object')


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,scale_factor_y,scale_factor_z,soma_surface,specimen_id,superseded,tags,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,0.1144,0.28,435.740270,478107198,False,3D Neuron Reconstruction morphology,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,0.1144,0.28,1446.587725,502367941,False,3D Neuron Reconstruction morphology,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,0.1144,0.28,287.118123,515771244,False,3D Neuron Reconstruction morphology,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,0.1144,0.28,180.994813,561435279,False,3D Neuron Reconstruction morphology,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,0.1144,0.28,55.055236,591268268,False,3D Neuron Reconstruction morphology,1659.465869,1185.773462,69.144146,aspiny


## Feature Engineering

In [123]:
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

df_full = df.drop(columns=['id','neuron_reconstruction_type','scale_factor_x', 'scale_factor_y','scale_factor_z','specimen_id','superseded','tags'])
print(df_full.shape)
df_full.head()

(619, 21)


,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,max_branch_order,max_euclidean_distance,max_path_distance,number_bifurcations,number_branches,...,number_stems,number_tips,overall_depth,overall_height,overall_width,soma_surface,total_length,total_surface,total_volume,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,6.0,99.779724,126.593790,33,73,...,7,40,51.4886,140.506829,136.267522,435.740270,1666.082926,1803.875644,167.343086,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,3.0,432.383110,496.831994,9,23,...,5,14,92.6671,566.701220,370.170045,1446.587725,2277.259374,4543.139073,921.571895,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,6.0,373.630444,436.958952,21,46,...,4,25,65.4696,425.897625,381.015114,287.118123,3589.339062,4704.910407,582.285423,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,11.0,943.382549,989.448317,24,52,...,4,28,99.9139,1217.694976,524.550156,180.994813,5416.228778,6814.933290,740.722806,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,5.0,186.218009,221.639502,14,32,...,4,18,54.3718,172.075941,261.459057,55.055236,1659.465869,1185.773462,69.144146,aspiny


In [124]:
X = abs(df_full.iloc[:,:-1]) # Need to take absolute value for SelectKBest to work
y = df_full.iloc[:,-1]       # iloc Purely integer-location based indexing for selection by position.

In [125]:
bestfeatures = SelectKBest(score_func=chi2)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df_full.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

print(featureScores.nlargest(20,'Score'))  #print 20 best features

                              Specs          Score
18                    total_surface  539939.690284
17                     total_length  264251.289979
10                     number_nodes  220597.718099
19                     total_volume  115805.409932
7                 max_path_distance   30418.233712
6            max_euclidean_distance   27774.577274
14                   overall_height   23066.856685
9                   number_branches    2410.349432
15                    overall_width    2214.133645
13                    overall_depth    1994.393610
12                      number_tips    1213.162125
8               number_bifurcations    1201.211153
16                     soma_surface     905.144986
5                  max_branch_order     556.265637
11                     number_stems      64.412472
3             average_fragmentation      55.236331
0   average_bifurcation_angle_local      16.120053
2                  average_diameter       0.834203
4     average_parent_daughter_r

In [126]:
pick_feats = list(featureScores.nlargest(20,'Score').Specs) # make a list of the ten best features
pick_feats.append('dendrite_type') # add dendrite_type to the list

df_small = df[pick_feats] # Make a new DataFrame with our selected features
df_small.head(5)

,total_surface,total_length,number_nodes,total_volume,max_path_distance,max_euclidean_distance,overall_height,number_branches,overall_width,overall_depth,...,number_bifurcations,soma_surface,max_branch_order,number_stems,average_fragmentation,average_bifurcation_angle_local,average_diameter,average_parent_daughter_ratio,average_contraction,dendrite_type
0,1803.875644,1666.082926,1470,167.343086,126.593790,99.779724,140.506829,73,136.267522,51.4886,...,33,435.740270,6.0,7,20.723077,82.727781,0.345092,0.964510,0.864267,aspiny
1,4543.139073,2277.259374,2011,921.571895,496.831994,432.383110,566.701220,23,370.170045,92.6671,...,9,1446.587725,3.0,5,105.277778,82.506680,0.634047,0.862183,0.903890,spiny
2,4704.910407,3589.339062,3137,582.285423,436.958952,373.630444,425.897625,46,381.015114,65.4696,...,21,287.118123,6.0,4,73.666667,77.536678,0.417929,0.926633,0.863104,spiny
3,6814.933290,5416.228778,4652,740.722806,989.448317,943.382549,1217.694976,52,524.550156,99.9139,...,24,180.994813,11.0,4,95.979167,76.583222,0.400396,0.942049,0.900537,spiny
4,1185.773462,1659.465869,1406,69.144146,221.639502,186.218009,172.075941,32,261.459057,54.3718,...,14,55.055236,5.0,4,47.535714,72.019250,0.227626,1.000000,0.873518,aspiny


### Now you have defined the training data set and the class labels. Next train the logistic regression classifier and the neural network like in the two examples and compare the performance of these two methods.
#### Example for logistic regression

In [127]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, scale
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Split the dataset into two groups for training and testing.

In [128]:
train = df_small.sample(frac=0.8,random_state=111) # random pick 80% for training
test = df_small.drop(train.index)
print(train.shape)
print(test.shape)

(495, 21)
(124, 21)


Train and test in different groups.

In [158]:
X = train.values[:,:-1] # put all columns from first to last but not including the last one in X
Y = train.values[:,-1]  # put last columns in y
x = test.values[:,:-1]  # put all columns from first to last but not including the last one in X
y = test.values[:,-1]   # put last columns in y

lb = LabelBinarizer()  # Binarize labels in a one-vs-all fashion
Y_b = lb.fit_transform(Y) # convert values in y into binary labels
y_b = lb.fit_transform(y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
training_y = encoder.transform(Y)
testing_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_training = np_utils.to_categorical(training_y)
dummy_testing = np_utils.to_categorical(testing_y)

log_reg = LogisticRegression(penalty="l2")
log_reg.fit(X,Y_b)

y_pred = log_reg.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))  # Calculate the accuracy comparing the predicted labels with the ground truth

Model accuracy: 0.8951612903225806


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### After you finish both logistic regression and neural network training on the morphology featrues, try to combine with the electrophysiology features and see how it will affect the result. You can find the corresponding cell in the electrophysiology features dataset according to the specimen_id.

### Neural Network for Morphology

In [159]:
def baseline_model():
    model = Sequential()
    model.add(Dense(22, input_dim=20, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 

model = baseline_model();
model.fit(X,dummy_training,epochs=100,verbose=1)
print("Model accuracy:", accuracy_score(y_b,y_pred)) 

Epoch 1/100
495/495 [==============================] - 1s 2ms/step - loss: 8.4848 - acc: 0.4707
Epoch 2/100
495/495 [==============================] - 0s 80us/step - loss: 8.7454 - acc: 0.4525
Epoch 3/100
495/495 [==============================] - 0s 98us/step - loss: 8.0190 - acc: 0.4970
Epoch 4/100
495/495 [==============================] - 0s 84us/step - loss: 8.2177 - acc: 0.4848
Epoch 5/100
495/495 [==============================] - 0s 82us/step - loss: 8.1287 - acc: 0.4929
Epoch 6/100
495/495 [==============================] - 0s 81us/step - loss: 8.3645 - acc: 0.4768
Epoch 7/100
495/495 [==============================] - 0s 80us/step - loss: 8.2684 - acc: 0.4828
Epoch 8/100
495/495 [==============================] - 0s 82us/step - loss: 7.9018 - acc: 0.5071
Epoch 9/100
495/495 [==============================] - 0s 78us/step - loss: 7.7722 - acc: 0.5152
Epoch 10/100
495/495 [==============================] - 0s 79us/step - loss: 7.7722 - acc: 0.5152
Epoch 11/100
495/495 [========

495/495 [==============================] - 0s 70us/step - loss: 7.7722 - acc: 0.5152
Epoch 85/100
495/495 [==============================] - 0s 69us/step - loss: 7.7722 - acc: 0.5152
Epoch 86/100
495/495 [==============================] - 0s 71us/step - loss: 7.7722 - acc: 0.5152
Epoch 87/100
495/495 [==============================] - 0s 71us/step - loss: 7.7722 - acc: 0.5152
Epoch 88/100
495/495 [==============================] - 0s 72us/step - loss: 7.7722 - acc: 0.5152
Epoch 89/100
495/495 [==============================] - 0s 68us/step - loss: 7.7722 - acc: 0.5152
Epoch 90/100
495/495 [==============================] - 0s 65us/step - loss: 7.7722 - acc: 0.5152
Epoch 91/100
495/495 [==============================] - 0s 63us/step - loss: 7.7722 - acc: 0.5152
Epoch 92/100
495/495 [==============================] - 0s 66us/step - loss: 7.7722 - acc: 0.5152
Epoch 93/100
495/495 [==============================] - 0s 66us/step - loss: 7.7722 - acc: 0.5152
Epoch 94/100
495/495 [===========

In [160]:
df_ef = pd.read_csv("ElecPhyFeatures.csv",index_col=0)       # Read electrophysiology data
print(df.shape)
print(df_ef.shape)

df_cb = pd.merge(df,df_ef,how='inner',left_on='specimen_id', right_on='specimen_id',suffixes=('_mp','')) # Combine two data frames
print(df_cb.shape)

df_cb = df_cb.drop(columns='dendrite_type_mp') # dendrite_type was duplicated while merging two dataframes, drop out one of them.
print(df_cb.shape)
print(df_cb.columns)
df_cb.head(5)

(619, 29)
(2333, 57)
(619, 85)
(619, 84)
Index(['average_bifurcation_angle_local', 'average_contraction',
       'average_diameter', 'average_fragmentation',
       'average_parent_daughter_ratio', 'id_mp', 'max_branch_order',
       'max_euclidean_distance', 'max_path_distance',
       'neuron_reconstruction_type', 'number_bifurcations', 'number_branches',
       'number_nodes', 'number_stems', 'number_tips', 'overall_depth',
       'overall_height', 'overall_width', 'scale_factor_x', 'scale_factor_y',
       'scale_factor_z', 'soma_surface', 'specimen_id', 'superseded', 'tags',
       'total_length', 'total_surface', 'total_volume', 'adaptation',
       'avg_isi', 'electrode_0_pa', 'f_i_curve_slope',
       'fast_trough_t_long_square', 'fast_trough_t_ramp',
       'fast_trough_t_short_square', 'fast_trough_v_long_square',
       'fast_trough_v_ramp', 'fast_trough_v_short_square', 'has_burst',
       'has_delay', 'has_pause', 'id', 'input_resistance_mohm', 'latency',
       'peak_t_lo

,average_bifurcation_angle_local,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,id_mp,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,...,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,dendrite_type
0,82.727781,0.864267,0.345092,20.723077,0.964510,491119743,6.0,99.779724,126.593790,dendrite-only,...,1.047640,-59.812500,-59.437504,-66.718750,2.625872,2.969025,1.909216,-77.718758,-63.952812,aspiny
1,82.506680,0.903890,0.634047,105.277778,0.862183,546781359,3.0,432.383110,496.831994,dendrite-only,...,1.114696,-58.125004,-57.864586,-66.443753,4.322536,4.272167,3.659369,-98.937508,-66.457214,spiny
2,77.536678,0.863104,0.417929,73.666667,0.926633,537042261,6.0,373.630444,436.958952,dendrite-only,...,1.236256,-56.000000,-57.979170,-69.356253,5.076478,5.069481,3.992479,-85.312500,-68.378876,spiny
3,76.583222,0.900537,0.400396,95.979167,0.942049,689123605,11.0,943.382549,989.448317,full,...,1.096650,-55.031250,-55.989587,-63.726564,3.445772,3.264711,3.118708,-83.625000,-62.545727,spiny
4,72.019250,0.873518,0.227626,47.535714,1.000000,657879305,5.0,186.218009,221.639502,full,...,1.377193,-60.656254,-54.562500,-72.192708,1.255817,1.051984,1.371654,-85.093750,-71.524811,aspiny


## Combined Features Logistic

In [161]:
df_cb = df_cb.drop(columns='neuron_reconstruction_type')
df_cb = df_cb.drop(columns='tags')
df_cb = df_cb.drop(columns='superseded')
df_cb.dropna(inplace=True)
combined_X = df_cb.iloc[:,:-1] # Need to take absolute value for SelectKBest to work
combined_y = df_cb.iloc[:,-1]       # iloc Purely integer-location based indexing for selection by position.

combined_train = df_cb.sample(frac=0.8,random_state=111) # random pick 80% for training
combined_test = df_cb.drop(combined_train.index)

combined_X = combined_train.values[:,:-1] # put all columns from first to last but not including the last one in X
combined_Y = combined_train.values[:,-1]  # put last columns in y
combined_x = combined_test.values[:,:-1]  # put all columns from first to last but not including the last one in X
combined_y = combined_test.values[:,-1]   # put last columns in y

combined_lb = LabelBinarizer()  # Binarize labels in a one-vs-all fashion
combined_Y_b = combined_lb.fit_transform(combined_Y) # convert values in y into binary labels
combined_y_b = combined_lb.fit_transform(combined_y)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(combined_y)
combined_training_y = encoder.transform(combined_Y)
combined_testing_y = encoder.transform(combined_y)
# convert integers to dummy variables (i.e. one hot encoded)
combined_dummy_training = np_utils.to_categorical(combined_training_y)
combined_dummy_testing = np_utils.to_categorical(combined_testing_y)

combined_log_reg = LogisticRegression(penalty="l2")
combined_log_reg.fit(combined_X,combined_Y_b)

combined_y_pred = combined_log_reg.predict(combined_x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred)) 

Model accuracy: 0.7368421052631579


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Combined Features Neural Network

In [162]:
def baseline_model():
    model = Sequential()
    model.add(Dense(83, input_dim=81, activation='relu'))
    model.add(Dense(83, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 

model = baseline_model();
model.fit(combined_X,combined_dummy_training,epochs=200,verbose=0)
combined_y_pred = model.predict(combined_x)
#print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred)) 

ValueError: Error when checking input: expected dense_70_input to have shape (81,) but got array with shape (80,)

# Additional Methods (Ensemble, etc) on just morphology

In [ ]:
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC

In [ ]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X,Y_b)

y_pred = rf.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred)) 

In [ ]:
et = ExtraTreesClassifier()
et.fit(X,Y_b)

y_pred = et.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

In [163]:
knn = KNeighborsClassifier()
knn.fit(X,Y_b)

y_pred = knn.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.8145161290322581


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [164]:
svc = SVC()
svc.fit(X,Y_b)

y_pred = svc.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.5564516129032258


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [165]:
rg = RidgeClassifier()
rg.fit(X,Y_b)

y_pred = rg.predict(x)  #Predict class labels for samples in X.

print("Model accuracy:", accuracy_score(y_b,y_pred))

Model accuracy: 0.9112903225806451


/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:839: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [166]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(combined_X,combined_Y_b)
y_pred1 = rf.predict(combined_x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred)) 

et = ExtraTreesClassifier()
et.fit(combined_X,combined_Y_b)
y_pred2 = et.predict(combined_x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred))

knn = KNeighborsClassifier()
knn.fit(combined_X,combined_Y_b)
y_pred3 = knn.predict(combined_x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred))

svc = SVC()
svc.fit(combined_X,combined_Y_b)
y_pred4 = svc.predict(x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred))

rg = RidgeClassifier()
rg.fit(combined_X,combined_Y_b)
y_pred5 = rg.predict(combined_x)  #Predict class labels for samples in X.
print("Model accuracy:", accuracy_score(combined_y_b,combined_y_pred))

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/opt/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/v

Model accuracy: 0.7368421052631579
Model accuracy: 0.7368421052631579
Model accuracy: 0.7368421052631579


ValueError: X.shape[1] = 20 should be equal to 80, the number of features at training time